In [1]:
import pandas as pd 
reviews = pd.read_csv('input/winemag-data-130k-v2.csv', index_col=0)
reviews['id'] = reviews.index
reviews.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,id
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,0
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,1
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,3
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,4


# 🛫 Data Filtering

In [4]:
'''
High score wine 
'''

reviews[['country', 'points']].loc[reviews['points'] > 90]
# reviews[reviews['points']>90][['country', 'points']] # same way to filtering data

,country,points
119,France,92
120,Italy,92
121,US,92
122,US,92
123,Australia,92
...,...,...
129936,France,91
129937,Austria,91
129938,Argentina,91
129939,Austria,91


In [3]:
'''
Select the wine with the score >90 and where country from Italy
'''
pd.DataFrame(reviews['id'].loc[(reviews['points'] > 90) & (reviews['country'] == 'Italy')])


,id
120,120
130,130
133,133
135,135
158,158
...,...
129675,129675
129681,129681
129892,129892
129893,129893


In [24]:
'''
Customers who never order
'''
customers = pd.DataFrame({'id':[1,2,3,4], 'name':['Joe', 'Henry', 'Sam', 'Max']})
orders = pd.DataFrame({'id':[1,2], 'customerId':[1,3]})

# pd.DataFrame({'Customers':customers[~customers.id.isin(orders.customerId)]['name']})

customers[~customers.id.isin(orders.customerId)]['name'].to_frame(name='Customers')

,Customers
1,Henry
3,Max


In [246]:
'''
Article Views I
找出看過自己文章的作者

+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| article_id    | int     |
| author_id     | int     |
| viewer_id     | int     |
| view_date     | date    |
+---------------+---------+

## Hint
* 同一位作者有多篇文章，所以只需要找 unique 的 id
'''

views = pd.DataFrame({
    'article_id':[1,1,2,2,4,3,3],
    'author_id':[3,3,7,7,7,4,4], 
    'viewer_id':[5,6,7,6,1,4,4]
    })

author_view = views.loc[views.author_id == views.viewer_id]['author_id'].unique()

author_view = sorted(author_view)

author_view = pd.DataFrame({'id':author_view})
author_view

,id
0,4
1,7


# 🥡String Methods

In [34]:
'''
Invalid text length 
找出描述超過15個字詞長度的id
'''
df = pd.DataFrame(reviews[reviews.description.apply(len)>15]['id'])

In [101]:
'''
Special bonus

酒的id為偶數且評酒人名字開頭為 V ，則評分為0
return id, points


## Hint
* 利用 str.startswith 的方法找字首開頭
'''

# Dealing with NaN
reviews.taster_name = reviews.taster_name.fillna("Unkown")

reviews['bonus'] = 0
reviews.loc[(reviews['id']%2 !=0) & ~(reviews['taster_name'].str.startswith('V')), 'bonus'] = reviews.points

special_bonus = reviews[['id', 'bonus']].sort_values(by='id', ascending=True)
special_bonus.head()

# Check result
# reviews.loc[reviews.taster_name.str.startswith('V')]

,id,bonus
0,0,0
1,1,87
2,2,0
3,3,87
4,4,0


In [44]:
'''
1667. Fix Names in a Table
修正姓名為僅有字首開頭為大寫

Input: 
Users table:
+---------+-------+ 
| user_id | name  |
+---------+-------+
| 1       | aLice |
| 2       | bOB   |
+---------+-------+
Output: 
+---------+-------+
| user_id | name  |
+---------+-------+
| 1       | Alice |
| 2       | Bob   |
+---------+-------+

## Hint
* 使用 str.capitalize 解決
'''

users = pd.DataFrame({'user_id':[1,2], 'name':['aLice', 'bOB']})
users['name'] = users['name'].str.capitalize()
users.sort_values(by='user_id', ascending=True)

,user_id,name
1,2,Bob
0,1,Alice


In [3]:
'''
A valid e-mail has a prefix name and a domain where:

The prefix name is a string that may contain letters (upper or lower case), digits, underscore '_', period '.', and/or dash '-'. The prefix name must start with a letter.
The domain is '@leetcode.com'.

Input: 
Users table:
+---------+-----------+-------------------------+
| user_id | name      | mail                    |
+---------+-----------+-------------------------+
| 1       | Winston   | winston@leetcode.com    |
| 2       | Jonathan  | jonathanisgreat         |
| 3       | Annabelle | bella-@leetcode.com     |
| 4       | Sally     | sally.come@leetcode.com |
| 5       | Marwan    | quarz#2020@leetcode.com |
| 6       | David     | david69@gmail.com       |
| 7       | Shapiro   | .shapo@leetcode.com     |
+---------+-----------+-------------------------+
Output: 
+---------+-----------+-------------------------+
| user_id | name      | mail                    |
+---------+-----------+-------------------------+
| 1       | Winston   | winston@leetcode.com    |
| 3       | Annabelle | bella-@leetcode.com     |
| 4       | Sally     | sally.come@leetcode.com |
+---------+-----------+-------------------------+
'''
import pandas as pd

users = pd.DataFrame({
    'user_id':[1,2,3,4,5,6,7],
    'name':['Winston', 'Jnathan', 'Annabelle', 'Sally', 'Marwan', 'David', 'Shapiro'],
    'mail':['winston@leetcode.com', 'jonathanisgreat', 'bella-@leetcode.com', 'sally.come@leetcode.com', 'quarz#2020@leetcode.com', 'david69@gmail.com', '.shapo@leetcode.com']
})

valid_email_df = users[users['mail'].str.match(r'b^[A-Za-z][A-Za-z0-9_\.-]*@leetcode\.com$')]
valid_email_df

,user_id,name,mail
0,1,Winston,winston@leetcode.com
2,3,Annabelle,bella-@leetcode.com
3,4,Sally,sally.come@leetcode.com


# 👩‍🍳Data Manipulation

In [182]:
''' Nth Highest Salary


## Hint
* 屏除薪資重複的資料
* 排序後取第N個 
'''
employee=pd.DataFrame({
    'id':[1,2,3],
    'salary':[100,100,300],
})

N=2
unique_salaries = employee['salary'].drop_duplicates()
if N-1 < len(unique_salaries.index):
    nth_salary = unique_salaries.sort_values(ascending=False).iloc[N-1]
    nth_highest_salary = {'getNthHighestSalary('+ str(N) +')':[nth_salary]}
else:
    nth_highest_salary = {'getNthHighestSalary('+ str(N) +')':[None]}
    
pd.DataFrame(nth_highest_salary)


,getNthHighestSalary(2)
0,100


In [71]:
'''
取第二高薪資 
'''
unique_salaries = employee['salary'].drop_duplicates()
if len(unique_salaries) < 2:
    second_high_salary = {'SecondHighestSalary':[None]}
else:
    second_highest = unique_salaries.sort_values(ascending=False).iloc[1]
    second_high_salary = {'SecondHighestSalary':[second_highest]}


pd.DataFrame(second_high_salary)


,SecondHighestSalary
0,100


In [212]:
'''
184. Department Highest Salary
返回各部門的最高薪資

Employee table:
+--------------+---------+
| Column Name  | Type    |
+--------------+---------+
| id           | int     |
| name         | varchar |
| salary       | int     |
| departmentId | int     |
+--------------+---------+

Department table:
+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| id          | int     |
| name        | varchar |
+-------------+---------+
'''

employee = pd.DataFrame({
    'id':[1,2,3,4,5],
    'name':['Joe', 'Jim', 'Henry', 'Sam', 'Max'],
    'salary':[70000, 90000, 80000, 60000, 90000],
    'departmentId':[1,1,2,2,1]
    })
department = pd.DataFrame({
    'id':[1,2],
    'name':['IT', 'Sales']
    })


# department = department.rename(columns={'id': 'departmentId', 'name':'Department'})
merged_df = employee.merge(department, left_on='departmentId', right_on='id', suffixes=('_employee', '_department'))
highest_salary_df = merged_df.groupby('departmentId').apply(lambda x: x[x['salary'] == x['salary'].max()])

highest_salary_df = highest_salary_df.reset_index(drop=True)
result_df = highest_salary_df[['name_department','name_employee', 'salary']]

result_df.columns = ['Department', 'Employee', 'Salary']
result_df


,Department,Employee,Salary
0,IT,Jim,90000
1,IT,Max,90000
2,Sales,Henry,80000


In [175]:
'''
178. Rank Scores
+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| id          | int     |
| score       | decimal |
+-------------+---------+
'''

scores = pd.DataFrame({'id':[1,2,3,4,5], 'score':[3.5, 4.0, 3.8, 4.0, 3.7]})

scores['rank'] = scores['score'].rank(method='dense', ascending=False)

scores[['score', 'rank']].sort_values(by='rank', ascending=True)

,score,rank
1,4.0,1.0
3,4.0,1.0
2,3.8,2.0
4,3.7,3.0
0,3.5,4.0


In [197]:
'''
刪除相同的資料，id 保留較小者
'''

person=pd.DataFrame({'id': [2,1,3], 'email':['abc@ecg.com', 'abc@ecg.com', 'hij@kml.com']})

person.sort_values(by='id', ascending=True, inplace=True)

person.drop_duplicates(subset='email', keep='first', inplace=True)
person

,id,email
1,1,abc@ecg.com
2,3,hij@kml.com


In [210]:
'''
1795. Rearrange Products Table

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| product_id  | int     |
| store1      | int     |
| store2      | int     |
| store3      | int     |
+-------------+---------+

pandas.melt 說明
frame : 你想要更動的 DataFrame。
id_vars: 可使用 tuple、list、或 ndarray，用以設定不想要被轉換的欄位。
value_vars: 可使用 tuple、list、或 ndarray，用以設定想要被拆解的欄位。 如果省略則拆解全部欄位。
var_name : 轉換後 id 的名稱。如果省略則設定為原本 DataFrame 的欄位名稱或是 variable。
value_name : 轉換後 value 欄位的名稱。如果省略則顯示原本 DataFrame 的欄位名稱或 value。
col_level : 可使用 int、string。如果 columns 是 MultiIndex，則使用該參數來進行選擇。

'''

products = pd.DataFrame({
    'product_id':[1,2],
    'store1':[95, 70],
    'store2': [100, None],
    'store3':[105,80]
})

pd.melt(products, id_vars='product_id', var_name='store', value_name='price').dropna()

,product_id,store,price
0,1,store1,95.0
1,2,store1,70.0
2,1,store2,100.0
4,1,store3,105.0
5,2,store3,80.0


# 📊 Statistics

In [229]:
'''
2082. The Number of Rich Customers

'''

store = pd.DataFrame({
    'bill_id':[6,8,4,11,13],
    'customer_id':[1,1,2,3,3],
    'amount':[549,834, 394, 657, 257],
    })

rich_customers = store[store['amount']>500]

num_rich_customers = rich_customers['customer_id'].nunique()

# pd.DataFrame({ 'rich_count':[len(store.groupby('customer_id').apply(lambda x: x[x['amount'] >500]).drop_duplicates(subset='customer_id')) ]} )

In [244]:
'''
1173. Immediate Food Delivery I

找出顧客期望收件日與實際收件日相同的計算及時送件的達成率(%)
6件中有2件及時送達，則達成率: 2/6*100% = 33.33

+-----------------------------+---------+
| Column Name                 | Type    |
+-----------------------------+---------+
| delivery_id                 | int     |
| customer_id                 | int     |
| order_date                  | date    |
| customer_pref_delivery_date | date    |
+-----------------------------+---------+

'''
delivery = pd.DataFrame({
    'order_date':['2019-08-01', '2019-08-02', '2019-08-11', '2019-08-24', '2019-08-21', '2019-08-11'],
    'customer_pref_delivery_date':['2019-08-02', '2019-08-02', '2019-08-11', '2019-08-26', '2019-08-22', '2019-08-13']
})

num_delivery =delivery.size
num_immediate_delivery = delivery[delivery['order_date'] == delivery['customer_pref_delivery_date']].size

pd.DataFrame({'immediate_percentage':[round((num_immediate_delivery/num_delivery)*100, 2)]})

,immediate_percentage
0,33.33


In [257]:
'''  1907. Count Salary Categories
"Low Salary": All the salaries strictly less than $20000.
"Average Salary": All the salaries in the inclusive range [$20000, $50000].
"High Salary": All the salaries strictly greater than $50000.

The result table must contain all three categories. If there are no accounts in a category, return 0.


'''
accounts = pd.DataFrame({
    'account_id':[3,2,8,6],
    'income':[120000, 12323, 84343, 94343]
})


pd.DataFrame({
    'category': ['Low Salary', 'Average Salary', 'High Salary'],
    'accounts_count': [
        accounts[accounts.income < 20000].shape[0],
        accounts[(accounts.income >= 20000) & (accounts.income <= 50000)].shape[0],
        accounts[accounts.income > 50000].shape[0],
    ],
})

,category,accounts_count
0,Low Salary,1
1,Average Salary,0
2,High Salary,3


# 🧮 Data Aggregation

In [282]:
''' 1741. Find Total Time Spent by Each Employee

* Note that within one day, an employee can enter and leave more than once. 
* The time spent in the office for a single entry is out_time - in_time.
'''

employees= pd.DataFrame({
    'emp_id':[1,1,1,2,2,],
    'event_day': ['2020-11-28', '2020-11-28', '2020-12-03', '2020-11-28', '2020-12-09'],
    'in_time': [4,55,1,3,47],
    'out_time': [32,200,42,33,74] 
})


employeees_agg_df = employees.groupby(['event_day', 'emp_id']).agg({'in_time': 'sum', 'out_time': 'sum'}).reset_index()
employeees_agg_df['total_time'] = employeees_agg_df['out_time'] - employeees_agg_df['in_time']

employeees_agg_df.rename(columns={'event_day': 'day'}, inplace=True)

employeees_agg_df[['day', 'emp_id', 'total_time']]

,day,emp_id,total_time
0,2020-11-28,1,173
1,2020-11-28,2,30
2,2020-12-03,1,41
3,2020-12-09,2,27


In [289]:
''' 
511. Game Play Analysis I
每個玩家第一次登入的時間
'''

activity=pd.DataFrame({
    'player_id':[1,1,2,3,3],
    'event_date':['2016-03-01', '2016-05-02', '2017-06-25', '2016-03-02', '2018-07-03']
})

first_login_day = pd.DataFrame(activity.groupby('player_id').event_date.min().reset_index())
first_login_day.rename(columns={'event_date':'first_login'}, inplace=True)
first_login_day

,player_id,first_login
0,1,2016-03-01
1,2,2017-06-25
2,3,2016-03-02


In [304]:
''' 2356. Number of Unique Subjects Taught by Each Teacher
+-------------+------+
| Column Name | Type |
+-------------+------+
| teacher_id  | int  |
| subject_id  | int  |
| dept_id     | int  |
+-------------+------+
'''

teacher = pd.DataFrame({
    'teacher_id':[1,1,1,2,2,2,2],
    'subject_id':[2,2,3,1,2,3,4],
    'dept_id':[3,4,3,1,1,1,1]
})

result_df = teacher.groupby(['teacher_id']).subject_id.nunique().to_frame().reset_index()
result_df.rename(columns={'subject_id':'cnt'}, inplace=True)

In [319]:
'''596. Classes More Than 5 Students
 
'''
courses = pd.DataFrame({
    'student':['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H',  'I'],
    'class':['Math', 'English', 'Math', 'Science', 'Math', 'Math', 'Math', 'Science', 'Math']
})

result_df = courses.groupby('class').student.size().reset_index()
result_df[result_df['student']>=5][['class']]

,class
1,Math


In [322]:
''' 586. Customer Placing the Largest Number of Orders
'''
orders=pd.DataFrame({
    'order_number':[1,2,3,4,5],
    'customer_number':[1,2,2,3,3]
})

result_df = orders.groupby('customer_number').count().reset_index()
result_df[result_df.order_number == result_df.order_number.max()][['customer_number']]

,customer_number
1,2
2,3


In [397]:
''' 1484. Group Sold Products By The Date
Activities table:
+------------+------------+
| sell_date  | product     |
+------------+------------+
| 2020-05-30 | Headphone  |
| 2020-06-01 | Pencil     |
| 2020-06-02 | Mask       |
| 2020-05-30 | Basketball |
| 2020-06-01 | Bible      |
| 2020-06-02 | Mask       |
| 2020-05-30 | T-Shirt    |
+------------+------------+
'''
activities = pd.DataFrame({
    'sell_date':['2020-05-30', '2020-06-01', '2020-06-02', '2020-05-30', '2020-06-01', '2020-06-02', '2020-05-30'],
    'product':['Headphone', 'Pencil', 'Mask', 'Basketball', 'Bible', 'Mask', 'T-Shirt']
})


'''
alternative solution 
Helper for column specific aggregation with control over output column names.

'''
sold_activities = activities.groupby('sell_date').agg(
        num_sold= pd.NamedAgg(column='product', aggfunc='nunique'),
        products = pd.NamedAgg(column='product', aggfunc=lambda x: ','.join(sorted(set(x))))
).reset_index()

''' Same solution'''
# sold_activities = activities.groupby('sell_date').agg(
#         ['nunique', 
#          lambda x: ','.join(sorted((set(x))))]
# ).reset_index()

# sold_activities.columns = ['sell_date', 'num_sold', 'products']

sold_activities


,sell_date,num_sold,products
0,2020-05-30,3,"Basketball,Headphone,T-Shirt"
1,2020-06-01,2,"Bible,Pencil"
2,2020-06-02,1,Mask


In [405]:
''' 1693. Daily Leads and Partners

DailySales table:
+-----------+-----------+---------+------------+
| date_id   | make_name | lead_id | partner_id |
+-----------+-----------+---------+------------+
| 2020-12-8 | toyota    | 0       | 1          |
| 2020-12-8 | toyota    | 1       | 0          |
| 2020-12-8 | toyota    | 1       | 2          |
| 2020-12-7 | toyota    | 0       | 2          |
| 2020-12-7 | toyota    | 0       | 1          |
| 2020-12-8 | honda     | 1       | 2          |
| 2020-12-8 | honda     | 2       | 1          |
| 2020-12-7 | honda     | 0       | 1          |
| 2020-12-7 | honda     | 1       | 2          |
| 2020-12-7 | honda     | 2       | 1          |
+-----------+-----------+---------+------------+
'''

daily_sales = pd.DataFrame({
    'date_id':['2020-12-8', '2020-12-8', '2020-12-8', '2020-12-7', '2020-12-7', '2020-12-8', '2020-12-8', '2020-12-7', '2020-12-7', '2020-12-7'],
    'make_name':['toyota', 'toyota', 'toyota', 'toyota', 'toyota', 'honda', 'honda', 'honda', 'honda', 'honda'],
    'lead_id':[0, 1, 1, 0, 0, 1, 2, 0, 1, 2],
    'partner_id':[1, 0, 2, 2, 1, 2, 1, 1, 2, 1]
})

result_df  = daily_sales.groupby(['date_id', 'make_name']).agg({
  'lead_id':'nunique',
  'partner_id':'nunique'  
}).reset_index()

result_df.columns = ['date_id', 'make_name', 'unique_leads', 'unique_partners']
result_df

,date_id,make_name,unique_leads,unique_partners
0,2020-12-7,honda,3,2
1,2020-12-7,toyota,1,2
2,2020-12-8,honda,2,2
3,2020-12-8,toyota,2,3


# Data integration

In [421]:
'''
1050. Actors and Directors Who Cooperated At Least Three Times

ActorDirector table:
+-------------+-------------+-------------+
| actor_id    | director_id | timestamp   |
+-------------+-------------+-------------+
| 1           | 1           | 0           |
| 1           | 1           | 1           |
| 1           | 1           | 2           |
| 1           | 2           | 3           |
| 1           | 2           | 4           |
| 2           | 1           | 5           |
| 2           | 1           | 6           |
+-------------+-------------+-------------+

'''

actor_director = pd.DataFrame({
    'actor_id':[1,1,1,1,1,2,2],
    'director_id':[1,1,1,2,2,1,1]
})

result_df = actor_director.groupby(['actor_id','director_id']).size().reset_index(name='pair_count')
result_df[result_df.pair_count >= 3][['actor_id', 'director_id']]

,actor_id,director_id
0,1,1


In [433]:
'''1378. Replace Employee ID With The Unique Identifier

Input: 
Employees table:
+----+----------+
| id | name     |
+----+----------+
| 1  | Alice    |
| 7  | Bob      |
| 11 | Meir     |
| 90 | Winston  |
| 3  | Jonathan |
+----+----------+
EmployeeUNI table:
+----+-----------+
| id | unique_id |
+----+-----------+
| 3  | 1         |
| 11 | 2         |
| 90 | 3         |
+----+-----------+
'''

employees = pd.DataFrame({
    'id':[1,7,11,90,3],
    'name':['Alice', 'Bob', 'Meir', 'Winston', 'Jonathan']
})
employee_uni = pd.DataFrame({
    'id':[3,11,90],
    'unique_id':[1,2,3]
})


employees.merge(employee_uni, on='id', how='left')[['unique_id', 'name']]

,unique_id,name
0,NaN,Alice
1,NaN,Bob
2,2.0,Meir
3,3.0,Winston
4,1.0,Jonathan


In [461]:
''' 
1280. Students and Examinations
Students table:
+------------+--------------+
| student_id | student_name |
+------------+--------------+
| 1          | Alice        |
| 2          | Bob          |
| 13         | John         |
| 6          | Alex         |
+------------+--------------+

subjects:
Subjects table:
+--------------+
| subject_name |
+--------------+
| Math         |
| Physics      |
| Programming  |
+--------------+

Examinations table:
+------------+--------------+
| student_id | subject_name |
+------------+--------------+
| 1          | Math         |
| 1          | Physics      |
| 1          | Programming  |
| 2          | Programming  |
| 1          | Physics      |
| 1          | Math         |
| 13         | Math         |
| 13         | Programming  |
| 13         | Physics      |
| 2          | Math         |
| 1          | Math         |
+------------+--------------+
'''
students = pd.DataFrame({
    'student_id':[1,2,13,6],
    'student_name':['Alice', 'Bob', 'John', 'Alex']
})


subjects=pd.DataFrame({
    'subject_name':['Math', 'Physics', 'Programming']
})

examinations=pd.DataFrame({
    'student_id':[1,1,1,2,1,1,13,13,13,2,1],
    'subject_name':['Math', 'Physics', 'Programming', 'Programming', 'Physics', 'Math', 'Math', 'Programming', 'Physics', 'Math', 'Math']
})


examination_count = examinations.groupby(['student_id', 'subject_name']).size().reset_index(name='attended_exams')
student_subjects = pd.merge(students, subjects,how='cross')
result_df = student_subjects.merge(examination_count, how='left', on=['student_id', 'subject_name'])\
    .fillna(0) \
    .sort_values(['student_id', 'subject_name'])
result_df[['student_id', 'student_name', 'subject_name', 'attended_exams']]


,student_id,student_name,subject_name,attended_exams
0,1,Alice,Math,3.0
1,1,Alice,Physics,2.0
2,1,Alice,Programming,1.0
3,2,Bob,Math,1.0
4,2,Bob,Physics,0.0
5,2,Bob,Programming,1.0
9,6,Alex,Math,0.0
10,6,Alex,Physics,0.0
11,6,Alex,Programming,0.0
6,13,John,Math,1.0


In [514]:
''' 570. Managers with at Least 5 Direct Reports
Input: 
Employee table:
+-----+-------+------------+-----------+
| id  | name  | department | managerId |
+-----+-------+------------+-----------+
| 101 | John  | A          | None      |
| 102 | Dan   | A          | 101       |
| 103 | James | A          | 101       |
| 104 | Amy   | A          | 101       |
| 105 | Anne  | A          | 101       |
| 106 | Ron   | B          | 101       |
+-----+-------+------------+-----------+
Output: 
+------+
| name |
+------+
| John |
+------+
 
'''
employee = pd.DataFrame({
    'id':[101,102,103,104,105,106],
    'name':['John', 'Dan', 'James', 'Amy', 'Anne', 'Ron'],
    'department':['A', 'A', 'A', 'A', 'A', 'B'],
    'managerId':[None, 101, 101, 101, 101, 101]
})


num_employee = employee.groupby(['managerId']).managerId.count().reset_index(name='direct_report')
num_employee = num_employee[num_employee.direct_report >= 5]

result_df = pd.merge(employee[['id', 'name']], num_employee, left_on='id', right_on='managerId', how='inner')
result_df[['name']]

,name
0,John


In [530]:
''' 607. Sales Person
Input: 
SalesPerson table:
+----------+------+--------+-----------------+------------+
| sales_id | name | salary | commission_rate | hire_date  |
+----------+------+--------+-----------------+------------+
| 1        | John | 100000 | 6               | 4/1/2006   |
| 2        | Amy  | 12000  | 5               | 5/1/2010   |
| 3        | Mark | 65000  | 12              | 12/25/2008 |
| 4        | Pam  | 25000  | 25              | 1/1/2005   |
| 5        | Alex | 5000   | 10              | 2/3/2007   |
+----------+------+--------+-----------------+------------+

Company table:
+--------+--------+----------+
| com_id | name   | city     |
+--------+--------+----------+
| 1      | RED    | Boston   |
| 2      | ORANGE | New York |
| 3      | YELLOW | Boston   |
| 4      | GREEN  | Austin   |
+--------+--------+----------+

Orders table:
+----------+------------+--------+----------+--------+
| order_id | order_date | com_id | sales_id | amount |
+----------+------------+--------+----------+--------+
| 1        | 1/1/2014   | 3      | 4        | 10000  |
| 2        | 2/1/2014   | 4      | 5        | 5000   |
| 3        | 3/1/2014   | 1      | 1        | 50000  |
| 4        | 4/1/2014   | 1      | 4        | 25000  |
+----------+------------+--------+----------+--------+
'''

sales_person = pd.DataFrame({
    'sales_id':[1,2,3,4,5],
    'name':['John', 'Amy', 'Mark', 'Pam', 'Alex'],
})


company = pd.DataFrame({
    'com_id':[1,2,3,4],
    'name':['RED', 'ORANGE', 'YELLOW', 'GREEN']
})

orders = pd.DataFrame({
    'order_id':[1,2,3,4],
    'com_id':[3,4,1,1],
    'sales_id':[4,5,1,4]
})

orders_company = pd.merge(orders, company, on='com_id', how='left')
orders_company.rename(columns={'name':'com_name'}, inplace=True)

# Get sales id in RED company.
red_sales_id = orders_company[orders_company.com_name == 'RED'].sales_id

# Find the sales person who hadn't order in RED company.
sales_person[~(sales_person.sales_id.isin(red_sales_id))].name.to_frame()
